## End-to-end agent

Example end to end agent:
```python
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")
```

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Define Tools

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Jakarta")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Jakarta', 'region': 'Jakarta Raya', 'country': 'Indonesia', 'lat': -6.21, 'lon': 106.85, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1721047365, 'localtime': '2024-07-15 19:42'}, 'current': {'last_updated_epoch': 1721046600, 'last_updated': '2024-07-15 19:30', 'temp_c': 28.1, 'temp_f': 82.6, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 300, 'wind_dir': 'WNW', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 50, 'feelslike_c': 29.7, 'feelslike_f': 85.4, 'windchill_c': 27.8, 'windchill_f': 82.0, 'heatindex_c': 29.2, 'heatindex_f': 84.6, 'dewpoint_c': 18.8, 'dewpoint_f': 65.9, 'vis_km': 4.4, 'vis_miles': 2.0, 'uv': 1.0, 'gust_mph': 10.0, 'gust_kph': 16.2}}"}, {'url': 'https://world-weather.info/forecast/indonesia/jakarta/july-2024/', 'conte

## Using Language Models

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [8]:
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_TVYbyTNFotIxKdzuCwNC3cgY', 'type': 'tool_call'}]


## Create the agent

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='987a580b-df41-4d73-9f41-6bc087a2c47b'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2f961d94-58fc-49c0-aad4-45d5574f98bc-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in hulu sungai tengah regency?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in hulu sungai tengah regency?', id='a855390a-b159-49dd-85de-d67dab17752f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vVHqe2kUZEgC6KtSFf9QMqG9', 'function': {'arguments': '{"query":"weather in Hulu Sungai Tengah Regency"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 95, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b4923388-c353-4642-9214-097de0c92462-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hulu Sungai Tengah Regency'}, 'id': 'call_vVHqe2kUZEgC6KtSFf9QMqG9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 27, 'total_tokens': 122}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Kampong Sungai Tengah\'

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in hulu sungai tengah regency?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dWlywMeJzv070qwUGo3God0J', 'function': {'arguments': '{"query":"weather in Hulu Sungai Tengah Regency"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 95, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9abf1688-22ab-4a42-ace1-9bf0a3fdc012-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hulu Sungai Tengah Regency'}, 'id': 'call_dWlywMeJzv070qwUGo3God0J', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 27, 'total_tokens': 122})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Kampong Sungai Tengah\', \'region\': \'\', \'country\': \'Singapore\', \'lat\': 1.38, \'

## Streaming tokens

In [16]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in hulu sungai tengah regency?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in Hulu Sungai Tengah Regency'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Kampong Sungai Tengah', 'region': '', 'country': 'Singapore', 'lat': 1.38, 'lon': 103.9, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1721047624, 'localtime': '2024-07-15 20:47'}, 'current': {'last_updated_epoch': 1721047500, 'last_updated': '2024-07-15 20:45', 'temp_c': 29.3, 'temp_f': 84.7, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 150, 'wind_dir': 'SSE', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 75, 'feelslike_c': 34.7, 'feelslike_f': 94.4, 'windchill_c': 27.9, 'windchill_f': 82.1, 'heatindex_c': 31.5, 'heatindex_f': 88.7, 'dewpoint_c': 23.4, 'dewpoint

## Adding in Memory

In [22]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [23]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Ihsan!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Ihsan! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 87, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5ee061c8-98b8-45bd-99b6-99f5d6bb6e91-0', usage_metadata={'input_tokens': 87, 'output_tokens': 13, 'total_tokens': 100})]}}
----


In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Ihsan. How can I help you further, Ihsan?', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 112, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bdca3a6e-735e-40b2-bae9-b9ca8784ffe6-0', usage_metadata={'input_tokens': 112, 'output_tokens': 19, 'total_tokens': 131})]}}
----


In [26]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal information like your name. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-da71d2ca-aa22-43f4-ba87-1f0ff9597dd8-0', usage_metadata={'input_tokens': 86, 'output_tokens': 20, 'total_tokens': 106})]}}
----
